How Python works:

* create AST
* create bytecode
* interpret bytecode

First, let's take a look at manually constructing an AST representing "1 + 2". 

Python is a much more complex language in terms of **Concrete Syntax**. It is not simple lists, like Scheme. It has brackets, newlines, colons, semicolons, dots, commas, and much more. Luckily for us, Python comes with a library to parse itself. To parse Python we import the `ast` module:

In [47]:
(import "ast")

(ast)

To use the built-in Python tools for interpreting an expression, we must build the AST in a precise manner. So, we take the idea of "1 + 2" and we express this as:

In [48]:
(define expr 
  (ast.Expression
    (ast.BinOp (ast.Num 1)
               (ast.Add)
               (ast.Num 2))))

In [49]:
(ast.dump expr)

"Expression(body=BinOp(left=Num(n=1), op=Add(), right=Num(n=2)))"

Because our expr didn't actually come from source code (e.g., from a file or a string) we need to poke in dummy values for line numbers so that the compiler won't complain:

In [50]:
(set! expr (ast.fix_missing_locations expr))

In Python, the AST is compiled into *bytecode*:

In [51]:
(define bytecode (compile expr "<string>" "eval"))

Finally, Python can interpret the bytecode and give you the result:

In [52]:
(python-eval bytecode)

3

In [53]:
(import "dis")

(dis)

In [54]:
(dis.disassemble bytecode)

  1           0 LOAD_CONST               2 (3)
              3 RETURN_VALUE


# Building a Python Interpreter

In [3]:
(import "ast")

(ast)

In [16]:
(define pyexpr (ast.parse "1 + 2"))

In [17]:
(ast.dump pyexpr)

"Module(body=[Expr(value=BinOp(left=Num(n=1), op=Add(), right=Num(n=2)))])"

We must handle a couple of items that the Python AST creates:

* Module
* Expr

Also note that `+` is parsed as a BinOp.

In [6]:
(define evaluator
  (lambda (expr)
    (cond
     ((isinstance expr ast.Module)
      (evaluator-begin (vector->list expr.body)))
     ((isinstance expr ast.Expr)
      (evaluator expr.value))
     ((isinstance expr ast.BinOp)
      (evaluator-binop expr.op 
                   (evaluator expr.left)
                   (evaluator expr.right)))
     ((isinstance expr ast.Num)
      expr.n)
     ((isinstance expr ast.Call)
      (evaluator-apply expr.func.id (map evaluator (vector->list expr.args))))
     (else (error "evaluator" "invalid ast: ~s" expr)))))

(define evaluator-apply
  (lambda (op operands)
    (cond
     ((string=? op "print")
      (apply print operands))
     (else (error "evaluator-appy" "unknown apply operator: ~s" op)))))

(define evaluator-begin
  (lambda (exprs)
    (cond
     ((null? exprs)
      (void))
     ((= 1 (length exprs))
      (evaluator (car exprs)))
     (else (begin
            (evaluator (car exprs))
            (evaluator-begin (cdr exprs)))))))

(define evaluator-binop
  (lambda (op left right)
    (cond
     ((isinstance op ast.Add)
      (+ left right))
     (else (error "apply-binop" "unknown operator: ~s" op)))))

In [18]:
(evaluator pyexpr)

3

## Python AST

Python can also, of course, handle the same AST that we can.

In [40]:
(define pyexpr (ast.parse "print(1 + 2); 4"))

In [41]:
(evaluator pyexpr)

3


4

In [42]:
(define bytecode (compile pyexpr "<string>" "exec"))

In [43]:
(python-eval bytecode)

3


**Exercise:** How did the Python interpreter differ from our interpretation of the same expression? Why do you thing this is?

In [44]:
(dis.disassemble bytecode)

  1           0 LOAD_NAME                0 (print)
              3 LOAD_CONST               3 (3)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 POP_TOP
             10 LOAD_CONST               2 (None)
             13 RETURN_VALUE


## P-Calc


In [11]:
(define pcalc
  (lambda (string)
    (let ((pyast (ast.parse string)))
      (evaluator pyast))))

In [13]:
(pcalc "(1 + 2)")

3

In [1]:
(try 
 (ast.parse "(+ 1 2)")
 (catch e e (error "evaluator" "Unable to parse")))
        


Traceback (most recent call last):
  File "In [1]", line 3, col 13, in 'error'
  File "In [1]", line 3, col 13
RunTimeError: Error in 'evaluator': Unable to parse



## Limitations

If we use the Python parser, then we can parse only valid Python strings. For example, this is invalid:

In [14]:
(pcalc "1 x 5")


Traceback (most recent call last):
  File "In [14]", line 1, col 1, in 'pcalc'
  File "In [11]", line 3, col 18, in 'ast.parse'
UnhandledException: invalid syntax (<unknown>, line 1)



That is, we can only use the symbols that Python has predeterimed as valid BinOps. If we want to define our own BinOp (not one of +, -, *, /, etc.) Then we must parse it as an application:

In [15]:
(pcalc "x(1, 5)")


Traceback (most recent call last):
  File "In [15]", line 1, col 1, in 'pcalc'
  File "In [11]", line 3, col 5, in 'let'
  File "In [11]", line 4, col 7, in 'evaluator'
  File "In [6]", line 5, col 7, in 'evaluator-begin'
  File "In [6]", line 31, col 7, in 'evaluator'
  File "In [6]", line 7, col 7, in 'evaluator'
  File "In [6]", line 15, col 7, in 'evaluator-apply'
  File "In [6]", line 23, col 12, in 'error'
  File "In [6]", line 23, col 12
RunTimeError: Error in 'evaluator-appy': unknown apply operator: "x"



Of course, we must handle the operand appropriately in evaluator-apply.